<a href="https://colab.research.google.com/github/smahensaria/MIDS_bridgeCourse_python/blob/master/Sundeep_Mahensaria_p2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project 2: Topic Classification

In this project, you'll work with text data from newsgroup posts on a variety of topics. You'll train classifiers to distinguish posts by topics inferred from the text. Whereas with digit classification, where each input is relatively dense (represented as a 28x28 matrix of pixels, many of which are non-zero), here each document is relatively sparse (represented as a bag-of-words). Only a few words of the total vocabulary are active in any given document. The assumption is that a label depends only on the count of words, not their order.

The `sklearn` documentation on feature extraction may be useful:
http://scikit-learn.org/stable/modules/feature_extraction.html

Each problem can be addressed succinctly with the included packages -- please don't add any more. Grading will be based on writing clean, commented code, along with a few short answers.

As always, you're welcome to work on the project in groups and discuss ideas on Slack, but <b> please prepare your own write-up with your own code. </b>

In [ ]:
# This tells matplotlib not to try opening a new window for each plot.
%matplotlib inline

# General libraries.
import re
import numpy as np
import matplotlib.pyplot as plt

# SK-learn libraries for learning.
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB

# SK-learn libraries for evaluation.
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import classification_report

# SK-learn library for importing the newsgroup data.
from sklearn.datasets import fetch_20newsgroups

# SK-learn libraries for feature extraction from text.
from sklearn.feature_extraction.text import *

import nltk

Load the data, stripping out metadata so that only textual features will be used, and restricting documents to 4 specific topics. By default, newsgroups data is split into training and test sets, but here the test set gets further split into development and test sets.  (If you remove the categories argument from the fetch function calls, you'd get documents from all 20 topics.)

In [ ]:
categories = ['alt.atheism', 'talk.religion.misc', 'comp.graphics', 'sci.space']
newsgroups_train = fetch_20newsgroups(subset='train',
                                      remove=('headers', 'footers', 'quotes'),
                                      categories=categories)
newsgroups_test  = fetch_20newsgroups(subset='test',
                                      remove=('headers', 'footers', 'quotes'),
                                      categories=categories)

num_test = int(len(newsgroups_test.target) / 2)
test_data, test_labels   = newsgroups_test.data[num_test:], newsgroups_test.target[num_test:]
dev_data, dev_labels     = newsgroups_test.data[:num_test], newsgroups_test.target[:num_test]
train_data, train_labels = newsgroups_train.data, newsgroups_train.target

print('training label shape:', train_labels.shape)
print('dev label shape:',      dev_labels.shape)
print('test label shape:',     test_labels.shape)
print('labels names:',         newsgroups_train.target_names)

### Part 1:

For each of the first 5 training examples, print the text of the message along with the label.

In [ ]:
#def P1(num_examples=5):
    ### STUDENT START ###
print('first 5 training examples - message and text:')
for i in np.arange(5):
  print('TRAINING EXAMPLE #', i+1, '. LABEL: ', newsgroups_train.target_names[train_labels[i]], '\n MESSAGE: ', train_data[i])
  print('\n')

    ### STUDENT END ###

#P1(5)

### Part 2:

Transform the training data into a matrix of **word** unigram feature vectors.  What is the size of the vocabulary? What is the average number of non-zero features per example?  What is the fraction of the non-zero entries in the matrix?  What are the 0th and last feature strings (in alphabetical order)?<br/>
_Use `CountVectorization` and its `.fit_transform` method.  Use `.nnz` and `.shape` attributes, and `.get_feature_names` method._

Now transform the training data into a matrix of **word** unigram feature vectors using your own vocabulary with these 4 words: ["atheism", "graphics", "space", "religion"].  Confirm the size of the vocabulary. What is the average number of non-zero features per example?<br/>
_Use `CountVectorization(vocabulary=...)` and its `.transform` method._

Now transform the training data into a matrix of **character** bigram and trigram feature vectors.  What is the size of the vocabulary?<br/>
_Use `CountVectorization(analyzer=..., ngram_range=...)` and its `.fit_transform` method._

Now transform the training data into a matrix of **word** unigram feature vectors and prune words that appear in fewer than 10 documents.  What is the size of the vocabulary?<br/>
_Use `CountVectorization(min_df=...)` and its `.fit_transform` method._

Now again transform the training data into a matrix of **word** unigram feature vectors. What is the fraction of words in the development vocabulary that is missing from the training vocabulary?<br/>
_Hint: Build vocabularies for both train and dev and look at the size of the difference._

Notes:
* `.fit_transform` makes 2 passes through the data: first it computes the vocabulary ("fit"), second it converts the raw text into feature vectors using the vocabulary ("transform").
* `.fit_transform` and `.transform` return sparse matrix objects.  See about them at http://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.sparse.csr_matrix.html.  

In [ ]:
#def P2():
    ### STUDENT START ###

# To create a Count Vectorizer, we simply need to instantiate one.
# There are special parameters we can set here when making the vectorizer, but
# for the most basic example, it is not needed.
vectorizer = CountVectorizer()

# To actually create the vectorizer, we simply need to call fit on the text
# data that we wish to fix
#vectorizer.fit(sample_text)
vectorizer.fit(train_data)

# Now, we can inspect how our vectorizer vectorized the text
# This will print out a list of words used, and their index in the vectors
print('Vocabulary: ')
print(vectorizer.vocabulary_)

# If we would like to actually create a vector, we can do so by passing the
# text into the vectorizer to get back counts
vector = vectorizer.transform(train_data)

# Our final vector:
print('Full vector: ')
print(vector.toarray())

    ### STUDENT END ###

#P2()

In [ ]:
#def P2():
    ### STUDENT START ###

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(train_data)
print(vectorizer.get_feature_names())

    ### STUDENT END ###

#P2()

What is the size of the vocabulary? \
What is the average number of non-zero features per example? \
What is the fraction of the non-zero entries in the matrix? \
What are the 0th and last feature strings (in alphabetical order)? \

Use CountVectorization and its `.fit_transform` method. Use `.nnz` and `.shape` attributes, and `.get_feature_names` method.

In [ ]:
print('Size of vocabulary:', len(vectorizer.get_feature_names())) # we can also use X.shape[1])

#Average number of non-zero features per example = number of non-zero features over all examples / number of examples
print('Average number of non-zero features per example:', X.nnz / (X.shape[0]))

#Fraction of non-zero entries in the matrix = number of non-zero entries / total number of entries
print('Fraction of non-zero entries in the matrix:', X.nnz / (X.shape[0]*X.shape[1]))

#0th and last feature strings (in alphabetical order)
print('0th string:', vectorizer.get_feature_names()[0], '; Last feature string:', vectorizer.get_feature_names()[len(vectorizer.get_feature_names())-1])

Now transform the training data into a matrix of **word** unigram feature vectors using your own vocabulary with these 4 words: ["atheism", "graphics", "space", "religion"].  \
Confirm the size of the vocabulary. \
What is the average number of non-zero features per example?<br/>
_Use `CountVectorization(vocabulary=...)` and its `.transform` method._

In [ ]:
myvocabulary =  ["atheism", "graphics", "space", "religion"]
vectorizer2 = CountVectorizer(vocabulary=myvocabulary)
X = vectorizer2.transform(train_data)
print(vectorizer2.get_feature_names())
print('Size of vocabulary:', len(vectorizer2.get_feature_names()))

print('Number of examples:', X.shape[0])
print('Average number of non-zero features per example:', X.nnz / X.shape[0])

Now transform the training data into a matrix of character bigram and trigram feature vectors. \
What is the size of the vocabulary? \
Use CountVectorization(analyzer=..., ngram_range=...) and its .fit_transform method.

In [ ]:
vectorizer3 = CountVectorizer(analyzer='char', ngram_range=(2,2))
X = vectorizer3.fit_transform(train_data)
#print(vectorizer3.get_feature_names())
print('Size of vocabulary for character bigram:', len(vectorizer3.get_feature_names()))

vectorizer4 = CountVectorizer(analyzer='char', ngram_range=(3,3))
X = vectorizer4.fit_transform(train_data)
print('Size of vocabulary for character trigram:', len(vectorizer4.get_feature_names()))

#let's try character unigram to se how many different characters have been used
vectorizer5 = CountVectorizer(analyzer='char', ngram_range=(1,1))
X = vectorizer5.fit_transform(train_data)
print('Size of vocabulary for character unigram:', len(vectorizer5.get_feature_names()))

Now transform the training data into a matrix of word unigram feature vectors and prune words that appear in fewer than 10 documents.\
What is the size of the vocabulary?\
Use CountVectorization(min_df=...) and its .fit_transform method.

In [ ]:
vectorizer6 = CountVectorizer(analyzer='word', ngram_range=(1,1), min_df=10)
X = vectorizer6.fit_transform(train_data)
print('Size of vocabulary without words that appear in fewer than 10 documents:', len(vectorizer6.get_feature_names()))

Now again transform the training data into a matrix of word unigram feature vectors.\
What is the fraction of words in the development vocabulary that is missing from the training vocabulary?\
Hint: Build vocabularies for both train and dev and look at the size of the difference.

In [ ]:
vectorizer7 = CountVectorizer()
X = vectorizer7.fit_transform(train_data)

print('Size of vocabulary in training data:', len(vectorizer7.get_feature_names()))

vectorizer8 = CountVectorizer()
X = vectorizer8.fit_transform(dev_data)

print('Size of vocabulary in development data:', len(vectorizer8.get_feature_names()))
print('Fraction of words in development data, missing from training data:', len(set(vectorizer8.get_feature_names()) - set(vectorizer7.get_feature_names()))/len(vectorizer8.get_feature_names()))

# #experimentation
# print('Vocabulary in development data, missing from training data:', len(set(vectorizer8.get_feature_names()) - set(vectorizer7.get_feature_names())))
# print('Vocabulary in development data, missing from training data:', (set(vectorizer8.get_feature_names()) - set(vectorizer7.get_feature_names())))
# print('Vocabulary in development data, missing from training data:', (set(vectorizer8.get_feature_names()).symmetric_difference(set(vectorizer7.get_feature_names()))))
# print('Vocabulary in development data, missing from training data (symmetric difference - WE DO NOT WANT THIS):', len(set(vectorizer8.get_feature_names()).symmetric_difference(set(vectorizer7.get_feature_names()))))

### Part 3:

Transform the training and development data to matrices of word unigram feature vectors.

1. Produce several k-Nearest Neigbors models by varying k, including one with k set to optimize f1 score.  For each model, show the k value and f1 score.
1. Produce several Naive Bayes models by varying smoothing (alpha), including one with alpha set approximately to optimize f1 score.  For each model, show the alpha value and f1 score.
1. Produce several Logistic Regression models by varying L2 regularization strength (C), including one with C set approximately to optimize f1 score.  For each model, show the C value, f1 score, and sum of squared weights for each topic.

* Why doesn't k-Nearest Neighbors work well for this problem?
* Why doesn't Logistic Regression work as well as Naive Bayes does?
* What is the relationship between logistic regression's sum of squared weights vs. C value?

Notes:
* Train on the transformed training data.
* Evaluate on the transformed development data.
* You can use `CountVectorizer` and its `.fit_transform` and `.transform` methods to transform data.
* You can use `KNeighborsClassifier(...)` to produce a k-Nearest Neighbors model.
* You can use `MultinomialNB(...)` to produce a Naive Bayes model.
* You can use `LogisticRegression(C=..., solver="liblinear", multi_class="auto")` to produce a Logistic Regression model.
* You can use `LogisticRegression`'s `.coef_` method to get weights for each topic.
* You can use `metrics.f1_score(..., average="weighted")` to compute f1 score.

In [ ]:
#def P3():
    ### STUDENT START ###
    ### STUDENT END ###

#P3()

ANSWER:

### Part 4:

Transform the data to a matrix of word **bigram** feature vectors.  Produce a Logistic Regression model.  For each topic, find the 5 features with the largest weights (that's 20 features in total).  Show a 20 row (features) x 4 column (topics) table of the weights.

Do you see any surprising features in this table?

Notes:
* Train on the transformed training data.
* You can use `CountVectorizer` and its `.fit_transform` method to transform data.
* You can use `LogisticRegression(C=0.5, solver="liblinear", multi_class="auto")` to produce a Logistic Regression model.
* You can use `LogisticRegression`'s `.coef_` method to get weights for each topic.
* You can use `np.argsort` to get indices sorted by element value. 

In [ ]:
#def P4():
    ### STUDENT START ###
    ### STUDENT END ###

#P4()

ANSWER:

### Part 5:

To improve generalization, it is common to try preprocessing text in various ways before splitting into words. For example, you could try transforming strings to lower case, replacing sequences of numbers with single tokens, removing various non-letter characters, and shortening long words.

Produce a Logistic Regression model (with no preprocessing of text).  Evaluate and show its f1 score and size of the dictionary.

Produce an improved Logistic Regression model by preprocessing the text.  Evaluate and show its f1 score and size of the vocabulary.  Try for an improvement in f1 score of at least 0.02.

How much did the improved model reduce the vocabulary size?

Notes:
* Train on the transformed training data.
* Evaluate on the transformed development data.
* You can use `CountVectorizer(preprocessor=...)` to preprocess strings with your own custom-defined function.
* `CountVectorizer` default is to preprocess strings to lower case.
* You can use `LogisticRegression(C=0.5, solver="liblinear", multi_class="auto")` to produce a logistic regression model.
* You can use `metrics.f1_score(..., average="weighted")` to compute f1 score.
* If you're not already familiar with regular expressions for manipulating strings, see https://docs.python.org/2/library/re.html, and re.sub() in particular.

In [ ]:
#def better_preprocessor(s):
    ### STUDENT START ###
    ### STUDENT END ###

#def P5():
    ### STUDENT START ###
    ### STUDENT END ###

#P5()

### Part 6:

The idea of regularization is to avoid learning very large weights (which are likely to fit the training data, but not generalize well) by adding a penalty to the total size of the learned weights. Logistic regression seeks the set of weights that minimizes errors in the training data AND has a small total size. The default L2 regularization computes this size as the sum of the squared weights (as in Part 3 above). L1 regularization computes this size as the sum of the absolute values of the weights. Whereas L2 regularization makes all the weights relatively small, L1 regularization drives many of the weights to 0, effectively removing unimportant features.

For several L1 regularization strengths ...<br/>
* Produce a Logistic Regression model using the **L1** regularization strength.  Reduce the vocabulary to only those features that have at least one non-zero weight among the four categories.  Produce a new Logistic Regression model using the reduced vocabulary and **L2** regularization strength of 0.5.  Evaluate and show the L1 regularization strength, vocabulary size, and f1 score associated with the new model.

Show a plot of f1 score vs. log vocabulary size.  Each point corresponds to a specific L1 regularization strength used to reduce the vocabulary.

How does performance of the models based on reduced vocabularies compare to that of a model based on the full vocabulary?

Notes:
* Train on the transformed training data.
* Evaluate on the transformed development data.
* You can use `LogisticRegression(..., penalty="l1")` to produce a logistic regression model using L1 regularization.
* You can use `LogisticRegression(..., penalty="l2")` to produce a logistic regression model using L2 regularization.
* You can use `LogisticRegression(..., tol=0.015)` to produce a logistic regression model using relaxed gradient descent convergence criteria.  The gradient descent code that trains the logistic regression model sometimes has trouble converging with extreme settings of the C parameter. Relax the convergence criteria by setting tol=.015 (the default is .0001).

In [ ]:
#def P6():
    # Keep this random seed here to make comparison easier.
    #np.random.seed(0)
    
    ### STUDENT START ###
    ### STUDENT END ###

#P6()

ANSWER:

### Part 7:

How is `TfidfVectorizer` different than `CountVectorizer`?

Produce a Logistic Regression model based on data represented in tf-idf form, with L2 regularization strength of 100.  Evaluate and show the f1 score.  How is `TfidfVectorizer` different than `CountVectorizer`?

Show the 3 documents with highest R ratio, where ...<br/>
$R\,ratio = maximum\,predicted\,probability \div predicted\,probability\,of\,correct\,label$

Explain what the R ratio describes.  What kinds of mistakes is the model making? Suggest a way to address one particular issue that you see.

Note:
* Train on the transformed training data.
* Evaluate on the transformed development data.
* You can use `TfidfVectorizer` and its `.fit_transform` method to transform data to tf-idf form.
* You can use `LogisticRegression(C=100, solver="liblinear", multi_class="auto")` to produce a logistic regression model.
* You can use `LogisticRegression`'s `.predict_proba` method to access predicted probabilities.

In [ ]:
#def P7():
    ### STUDENT START ###
    ### STUDENT END ###

#P7()

ANSWER:

### Part 8 EXTRA CREDIT:

Produce a Logistic Regression model to implement your suggestion from Part 7.